In [1]:
# Cell 1: imports, config, helper functions
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from math import sqrt
from sqlalchemy import create_engine, text

# Constants / config
TRADING_DAYS = 252  # annualization factor
MASTER_CSV = "../data/master_table.csv"

# Helper: safe CAGR between two dates (returns decimal, e.g. 0.123 for 12.3%)
def compute_cagr_from_series(nav_series, dates_series):
    """
    nav_series: pd.Series of NAV values sorted by date ascending
    dates_series: pd.Series of corresponding pd.Timestamp
    Returns CAGR in decimal. If period < 1 year returns np.nan (use absolute return instead).
    """
    if len(nav_series) < 2:
        return np.nan
    start_nav = float(nav_series.iloc[0])
    end_nav = float(nav_series.iloc[-1])
    days = (pd.Timestamp(dates_series.iloc[-1]) - pd.Timestamp(dates_series.iloc[0])).days
    if days <= 0:
        return np.nan
    years = days / 365.25
    if years < 1:
        return np.nan
    return (end_nav / start_nav) ** (1.0 / years) - 1.0

# Helper: XIRR-style SIP already exists in your repo; we will use simplified monthly-first-day SIP
def compute_sip_units(first_of_month_navs, sip_amount=1000):
    """
    first_of_month_navs: pd.Series of NAV indexed by date (ascending)
    Returns: pandas.Series of cumulative value of SIP invested each date (value at each NAV date)
    """
    units_each = sip_amount / first_of_month_navs
    cum_units = units_each.cumsum()
    value_series = cum_units * first_of_month_navs
    return value_series, units_each, cum_units


In [2]:
# Cell 2: load and basic cleaning (re-uses your conventions)
df = pd.read_csv(MASTER_CSV, parse_dates=['date'])
df.rename(columns={'date':'date', 'nav':'nav'}, inplace=True)  # ensure names
df = df.sort_values(['fund_name','date']).reset_index(drop=True)

# Ensure numeric nav and drop rows without nav
df['nav'] = pd.to_numeric(df['nav'], errors='coerce')
df = df.dropna(subset=['nav', 'fund_name', 'date'])
# Recompute daily returns as decimal (not percent)
df['daily_return'] = df.groupby('fund_name')['nav'].pct_change()


In [3]:
print("Funds loaded:", df['fund_name'].nunique())
print("Rows:", len(df))
print("Date range:", df['date'].min(), "→", df['date'].max())
print("Sample daily returns:")
display(df.groupby('fund_name').head(3))

Funds loaded: 8
Rows: 36929
Date range: 2013-01-01 00:00:00 → 2025-10-29 00:00:00
Sample daily returns:


,date,fund_name,nav,cap,fund_type,daily_return
0,2013-01-02,DSP Midcap Fund,20.5420,Mid Cap,Equity,NaN
1,2013-01-03,DSP Midcap Fund,20.6190,Mid Cap,Equity,0.003748
2,2013-01-04,DSP Midcap Fund,20.6480,Mid Cap,Equity,0.001406
4684,2013-01-01,HDFC Corporate Bond Fund,12.4504,Debt,Debt,NaN
4685,2013-01-02,HDFC Corporate Bond Fund,12.4584,Debt,Debt,0.000643
4686,2013-01-03,HDFC Corporate Bond Fund,12.4678,Debt,Debt,0.000755
9369,2014-06-30,HDFC Large and Mid Cap Fund,82.1250,Large & Mid Cap,Equity,NaN
9370,2014-07-01,HDFC Large and Mid Cap Fund,82.1710,Large & Mid Cap,Equity,0.000560
9371,2014-07-02,HDFC Large and Mid Cap Fund,82.9860,Large & Mid Cap,Equity,0.009918
13509,2013-01-02,ICICI Prudential Balanced Advantage Fund,17.4000,Multi / Hybrid,Balanced / Hybrid,NaN


In [4]:
# Cell 3: risk-free rate selection (annual decimal)
# Industry standard: use 10-year Government of India bond yield as the risk-free rate for equity analysis.
# You can override rf_annual if you want another number or historic average.
# Based on market data around Oct 2025 the 10Y yield is about 6.5% (0.065) — adjust if you want the latest.
rf_annual = 0.065   # 6.5% as decimal; replace if you prefer to fetch a different value programmatically
print(f"[INFO] Using risk-free (annual) = {rf_annual*100:.2f}% (10Y GOI proxy).")

[INFO] Using risk-free (annual) = 6.50% (10Y GOI proxy).


In [5]:
# Cell 4: cumulative return series per fund (decimal internally, then convert to % in-place)
cum_frames = []
for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy()
    
    # Keep full precision daily return for cumulative calculation
    daily_ret_full = g['daily_return'].copy().fillna(0)  # decimal
    
    # Cumulative return in decimal using full precision
    g['cum_return'] = (1 + daily_ret_full).cumprod() - 1
    g.loc[g['daily_return'].isna(), 'cum_return'] = 0.0
    
    # Convert daily_return (already rounded for display) and cum_return to percentages
    g['daily_return'] = (g['daily_return'] * 100).round(2)
    g['cum_return'] = (g['cum_return'] * 100).round(2)
    
    cum_frames.append(g[['fund_name','date','nav','daily_return','cum_return']])

cum_df = pd.concat(cum_frames, ignore_index=True)
# show sample
display(cum_df.groupby('fund_name').head(3))


,fund_name,date,nav,daily_return,cum_return
0,DSP Midcap Fund,2013-01-02,20.5420,NaN,0.00
1,DSP Midcap Fund,2013-01-03,20.6190,0.37,0.37
2,DSP Midcap Fund,2013-01-04,20.6480,0.14,0.52
4684,HDFC Corporate Bond Fund,2013-01-01,12.4504,NaN,0.00
4685,HDFC Corporate Bond Fund,2013-01-02,12.4584,0.06,0.06
4686,HDFC Corporate Bond Fund,2013-01-03,12.4678,0.08,0.14
9369,HDFC Large and Mid Cap Fund,2014-06-30,82.1250,NaN,0.00
9370,HDFC Large and Mid Cap Fund,2014-07-01,82.1710,0.06,0.06
9371,HDFC Large and Mid Cap Fund,2014-07-02,82.9860,0.99,1.05
13509,ICICI Prudential Balanced Advantage Fund,2013-01-02,17.4000,NaN,0.00


In [6]:
# Cell 5: Volatility per fund — industry-standard horizons: 1Y, 3Y, 5Y trailing
vol_horizons = [1, 3, 5]  # years
vol_records = []

for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy()
    last_date = g['date'].max()
    
    for yr in vol_horizons:
        start_date = last_date - pd.DateOffset(years=yr)
        series = g[g['date'] > start_date]['daily_return'].dropna()
        
        if series.empty or len(series) < 2:
            vol_ann = np.nan
        else:
            # Annualized volatility in %, decimal daily_return × √252
            vol_ann = series.std(ddof=1) * sqrt(TRADING_DAYS) * 100
        
        vol_records.append({
            'fund_name': fund,
            'horizon': f'{yr}Y',
            'volatility_annual': round(vol_ann, 2)
        })

vol_df = pd.DataFrame(vol_records).set_index(['fund_name','horizon'])
display(vol_df)


volatility_annual
fund_name                                          horizon                   
DSP Midcap Fund                                    1Y                   13.34
                                                   3Y                   11.70
                                                   5Y                   12.19
HDFC Corporate Bond Fund                           1Y                    1.02
                                                   3Y                    0.77
                                                   5Y                    0.92
HDFC Large and Mid Cap Fund                        1Y                   12.01
                                                   3Y                   11.12
                                                   5Y                   12.55
ICICI Prudential Balanced Advantage Fund           1Y                    4.98
                                                   3Y                    4.37
                                                   5Y                    4.78
ICICI Prudential Large Cap Fund (erstwhile Blue... 1Y                    9.70
                                                   3Y                    9.30
                                                   5Y                   10.86
Nippon India Small Cap Fund                        1Y                   13.94
                                                   3Y                   12.60
                                                   5Y                   13.61
SBI Large & Midcap Fund                            1Y                   10.40
                                                   3Y                    9.38
                                                   5Y                   10.72
UTI Nifty 50 Index Fund                            1Y                   10.40
                                                   3Y                    9.96
                                                   5Y                   11.65

In [7]:
# ===============================
# Cell 6: Compute Sharpe for several horizons and Since Inception -->loaded to db
# Industry-standard approach using CAGR logic
# ===============================
from dateutil.relativedelta import relativedelta

horizons = [1, 3, 5, 10]  # years
sharpe_records = []

for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy()
    last_date = g['date'].max()
    
    # ------------------
    # Since Inception
    # ------------------
    n_years = (last_date - g['date'].iloc[0]).days / 365.25
    if n_years < 1:
        cagr_si = (g['nav'].iloc[-1] / g['nav'].iloc[0]) - 1.0
    else:
        cagr_si = (g['nav'].iloc[-1] / g['nav'].iloc[0]) ** (1/n_years) - 1.0
    
    vol_si = g['daily_return'].dropna().std(ddof=1) * sqrt(TRADING_DAYS) if len(g['daily_return'].dropna())>1 else np.nan
    sharpe_si = (cagr_si - rf_annual) / vol_si if (not np.isnan(cagr_si) and vol_si and vol_si>0) else np.nan

    sharpe_records.append({
        'fund_name': fund,
        'horizon': 'Since Inception',
        'cagr': cagr_si,
        'volatility': vol_si,
        'sharpe': sharpe_si
    })

    # ------------------
    # Horizon-specific
    # ------------------
    for yr in horizons:
        start_date = last_date - relativedelta(years=yr)
        sub = g[g['date'] >= start_date].copy()
        
        # Include previous NAV if <2 points
        if len(sub) < 2:
            prev = g[g['date'] < start_date].tail(1)
            if not prev.empty:
                sub = pd.concat([prev, sub])
        
        if len(sub) < 2:
            cagr = np.nan
            vol = np.nan
            sharpe = np.nan
        else:
            # Compute number of years
            n_years_h = (sub['date'].iloc[-1] - sub['date'].iloc[0]).days / 365.25
            if n_years_h < 1:
                # absolute return for <1 year
                cagr = (sub['nav'].iloc[-1] / sub['nav'].iloc[0]) - 1.0
            else:
                # CAGR for ≥1 year
                cagr = (sub['nav'].iloc[-1] / sub['nav'].iloc[0]) ** (1/n_years_h) - 1.0
            
            vol = sub['daily_return'].dropna().std(ddof=1) * sqrt(TRADING_DAYS) if len(sub['daily_return'].dropna())>1 else np.nan
            sharpe = (cagr - rf_annual) / vol if (not np.isnan(cagr) and vol and vol>0) else np.nan
        
        sharpe_records.append({
            'fund_name': fund,
            'horizon': f'{yr}Y',
            'cagr': cagr,
            'volatility': vol,
            'sharpe': sharpe
        })

# ------------------
# Convert to % and round
# ------------------
sharpe_df = pd.DataFrame(sharpe_records).set_index(['fund_name','horizon'])
sharpe_df['cagr'] = (sharpe_df['cagr'])
sharpe_df['volatility'] = (sharpe_df['volatility'])
sharpe_df['sharpe'] = sharpe_df['sharpe'].round(2)

display(sharpe_df.groupby(level=0).head(10))


cagr  \
fund_name                                          horizon                     
DSP Midcap Fund                                    Since Inception  0.177979   
                                                   1Y               0.060378   
                                                   3Y               0.209633   
                                                   5Y               0.207182   
                                                   10Y              0.162082   
HDFC Corporate Bond Fund                           Since Inception  0.081478   
                                                   1Y               0.081669   
                                                   3Y               0.082095   
                                                   5Y               0.064887   
                                                   10Y              0.078337   
HDFC Large and Mid Cap Fund                        Since Inception  0.142543   
                                                   1Y               0.070610   
                                                   3Y               0.224592   
                                                   5Y               0.280506   
                                                   10Y              0.156621   
ICICI Prudential Balanced Advantage Fund           Since Inception  0.132602   
                                                   1Y               0.109848   
                                                   3Y               0.143173   
                                                   5Y               0.155750   
                                                   10Y              0.122571   
ICICI Prudential Large Cap Fund (erstwhile Blue... Since Inception  0.160873   
                                                   1Y               0.081400   
                                                   3Y               0.193262   
                                                   5Y               0.230288   
                                                   10Y              0.156625   
Nippon India Small Cap Fund                        Since Inception  0.250459   
                                                   1Y               0.002920   
                                                   3Y               0.244237   
                                                   5Y               0.341429   
                                                   10Y              0.216759   
SBI Large & Midcap Fund                            Since Inception  0.174143   
                                                   1Y               0.080974   
                                                   3Y               0.182992   
                                                   5Y               0.250197   
                                                   10Y              0.160288   
UTI Nifty 50 Index Fund                            Since Inception  0.131326   
                                                   1Y               0.076158   
                                                   3Y               0.146147   
                                                   5Y               0.185591   
                                                   10Y              0.134577   

                                                                    volatility  \
fund_name                                          horizon                       
DSP Midcap Fund                                    Since Inception    0.132669   
                                                   1Y                 0.133422   
                                                   3Y                 0.116945   
                                                   5Y                 0.121848   
                                                   10Y                0.128470   
HDFC Corporate Bond Fund                           Since Inception    0.014679   
                                                   1Y  

In [8]:
# ------------------------------
# Cell 7: Sortino ratio for several horizons and Since Inception -->loaded to db
# ------------------------------
from dateutil.relativedelta import relativedelta

horizons = [1, 3, 5, 10]  # years
sortino_records = []

def annualized_downside_std(daily_returns, mar_daily):
    """Annualized downside standard deviation"""
    daily_returns = daily_returns.dropna()
    if len(daily_returns) == 0:
        return np.nan
    diff = daily_returns - mar_daily
    downside = diff[diff < 0]
    if len(downside) == 0:
        return 0.0
    return downside.std(ddof=1) * np.sqrt(TRADING_DAYS)

mar_daily = rf_annual / TRADING_DAYS

for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy()
    last_date = g['date'].max()
    
    # ------------------
    # Since Inception
    # ------------------
    n_years = (last_date - g['date'].iloc[0]).days / 365.25
    cagr_si = (g['nav'].iloc[-1] / g['nav'].iloc[0]) ** (1/n_years) - 1.0 if n_years >= 1 else (g['nav'].iloc[-1] / g['nav'].iloc[0]) - 1.0
    down_std_si = annualized_downside_std(g['daily_return'], mar_daily)
    sortino_si = (cagr_si - rf_annual) / down_std_si if (not np.isnan(cagr_si) and down_std_si and down_std_si > 0) else np.nan

    sortino_records.append({
        'fund_name': fund,
        'horizon': 'Since Inception',
        'cagr': cagr_si,
        'downside_std': down_std_si,
        'sortino': sortino_si
    })

    # ------------------
    # Horizon-specific
    # ------------------
    for yr in horizons:
        start_date = last_date - relativedelta(years=yr)
        sub = g[g['date'] >= start_date].copy()
        
        # Include previous NAV if <2 points
        if len(sub) < 2:
            prev = g[g['date'] < start_date].tail(1)
            if not prev.empty:
                sub = pd.concat([prev, sub])
        
        if len(sub) < 2:
            cagr_h = np.nan
            down_std_h = np.nan
            sortino_h = np.nan
        else:
            n_years_h = (sub['date'].iloc[-1] - sub['date'].iloc[0]).days / 365.25
            cagr_h = (sub['nav'].iloc[-1] / sub['nav'].iloc[0]) ** (1/n_years_h) - 1.0 if n_years_h >= 1 else (sub['nav'].iloc[-1] / sub['nav'].iloc[0]) - 1.0
            down_std_h = annualized_downside_std(sub['daily_return'], mar_daily)
            sortino_h = (cagr_h - rf_annual) / down_std_h if (not np.isnan(cagr_h) and down_std_h and down_std_h > 0) else np.nan
        
        sortino_records.append({
            'fund_name': fund,
            'horizon': f'{yr}Y',
            'cagr': cagr_h,
            'downside_std': down_std_h,
            'sortino': sortino_h
        })

sortino_df = pd.DataFrame(sortino_records).set_index(['fund_name','horizon'])

# Convert CAGR and downside std to % and round 2 decimals; Sortino unitless
sortino_df['cagr'] = (sortino_df['cagr'])
sortino_df['downside_std'] = (sortino_df['downside_std'])
sortino_df['sortino'] = sortino_df['sortino'].round(2)

display(sortino_df.groupby(level=0).head(10))


cagr  \
fund_name                                          horizon                     
DSP Midcap Fund                                    Since Inception  0.177979   
                                                   1Y               0.060378   
                                                   3Y               0.209633   
                                                   5Y               0.207182   
                                                   10Y              0.162082   
HDFC Corporate Bond Fund                           Since Inception  0.081478   
                                                   1Y               0.081669   
                                                   3Y               0.082095   
                                                   5Y               0.064887   
                                                   10Y              0.078337   
HDFC Large and Mid Cap Fund                        Since Inception  0.142543   
                                                   1Y               0.070610   
                                                   3Y               0.224592   
                                                   5Y               0.280506   
                                                   10Y              0.156621   
ICICI Prudential Balanced Advantage Fund           Since Inception  0.132602   
                                                   1Y               0.109848   
                                                   3Y               0.143173   
                                                   5Y               0.155750   
                                                   10Y              0.122571   
ICICI Prudential Large Cap Fund (erstwhile Blue... Since Inception  0.160873   
                                                   1Y               0.081400   
                                                   3Y               0.193262   
                                                   5Y               0.230288   
                                                   10Y              0.156625   
Nippon India Small Cap Fund                        Since Inception  0.250459   
                                                   1Y               0.002920   
                                                   3Y               0.244237   
                                                   5Y               0.341429   
                                                   10Y              0.216759   
SBI Large & Midcap Fund                            Since Inception  0.174143   
                                                   1Y               0.080974   
                                                   3Y               0.182992   
                                                   5Y               0.250197   
                                                   10Y              0.160288   
UTI Nifty 50 Index Fund                            Since Inception  0.131326   
                                                   1Y               0.076158   
                                                   3Y               0.146147   
                                                   5Y               0.185591   
                                                   10Y              0.134577   

                                                                    downside_std  \
fund_name                                          horizon                         
DSP Midcap Fund                                    Since Inception      0.110724   
                                                   1Y                   0.106453   
                                                   3Y                   0.096840   
                                                   5Y                   0.100398   
                                                   10Y                  0.109133   
HDFC Corporate Bond Fund                           Since Inception      0.011385   
                                       

In [9]:
# Cell 8: Rolling vol (annualized) for selected windows (21, 60, 252 days)
rolling_windows = [21, 60, 252]
rolling_vols = []

for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy()
    g['fund_name'] = fund  # Add fund_name column
    for w in rolling_windows:
        colname = f'rolling_vol_{w}d'
        g[colname] = g['daily_return'].rolling(window=w, min_periods=int(w/2)).std(ddof=1) * sqrt(TRADING_DAYS)
    rolling_vols.append(g[['fund_name', 'date'] + [f'rolling_vol_{w}d' for w in rolling_windows]])

# Combine all funds into a single DataFrame
rolling_vol_df = pd.concat(rolling_vols, ignore_index=True)

# Display tail sample for each fund
for fund, sub in rolling_vol_df.groupby('fund_name'):
    print(f"Rolling vol sample for fund: {fund}")
    display(sub.tail(5))


Rolling vol sample for fund: DSP Midcap Fund


,fund_name,date,rolling_vol_21d,rolling_vol_60d,rolling_vol_252d
4679,DSP Midcap Fund,2025-10-25,0.062865,0.080288,0.123350
4680,DSP Midcap Fund,2025-10-26,0.062865,0.080288,0.123350
4681,DSP Midcap Fund,2025-10-27,0.070841,0.080842,0.123781
4682,DSP Midcap Fund,2025-10-28,0.070770,0.080543,0.123758
4683,DSP Midcap Fund,2025-10-29,0.066960,0.080793,0.123438


Rolling vol sample for fund: HDFC Corporate Bond Fund


,fund_name,date,rolling_vol_21d,rolling_vol_60d,rolling_vol_252d
9364,HDFC Corporate Bond Fund,2025-10-25,0.006329,0.007480,0.011419
9365,HDFC Corporate Bond Fund,2025-10-26,0.006329,0.007480,0.011419
9366,HDFC Corporate Bond Fund,2025-10-27,0.005213,0.006623,0.011416
9367,HDFC Corporate Bond Fund,2025-10-28,0.004738,0.006180,0.011416
9368,HDFC Corporate Bond Fund,2025-10-29,0.004554,0.006158,0.011414


Rolling vol sample for fund: HDFC Large and Mid Cap Fund


...

,fund_name,date,rolling_vol_21d,rolling_vol_60d,rolling_vol_252d
27556,Nippon India Small Cap Fund,2025-10-25,0.055390,0.073768,0.124660
27557,Nippon India Small Cap Fund,2025-10-26,0.055390,0.073768,0.124660
27558,Nippon India Small Cap Fund,2025-10-27,0.056958,0.071914,0.124648
27559,Nippon India Small Cap Fund,2025-10-28,0.056641,0.071503,0.124207
27560,Nippon India Small Cap Fund,2025-10-29,0.057440,0.072478,0.123638


Rolling vol sample for fund: SBI Large & Midcap Fund


,fund_name,date,rolling_vol_21d,rolling_vol_60d,rolling_vol_252d
32240,SBI Large & Midcap Fund,2025-10-25,0.053340,0.064262,0.099657
32241,SBI Large & Midcap Fund,2025-10-26,0.053340,0.064262,0.099657
32242,SBI Large & Midcap Fund,2025-10-27,0.057018,0.063480,0.099861
32243,SBI Large & Midcap Fund,2025-10-28,0.057053,0.062666,0.099828
32244,SBI Large & Midcap Fund,2025-10-29,0.051378,0.062855,0.099611


Rolling vol sample for fund: UTI Nifty 50 Index Fund


,fund_name,date,rolling_vol_21d,rolling_vol_60d,rolling_vol_252d
36924,UTI Nifty 50 Index Fund,2025-10-25,0.060460,0.062669,0.101960
36925,UTI Nifty 50 Index Fund,2025-10-26,0.060460,0.062669,0.101960
36926,UTI Nifty 50 Index Fund,2025-10-27,0.060407,0.061057,0.102183
36927,UTI Nifty 50 Index Fund,2025-10-28,0.061202,0.060651,0.102190
36928,UTI Nifty 50 Index Fund,2025-10-29,0.060124,0.061035,0.102262


In [10]:
# Cell 9: Growth of ₹1000 invested (Lumpsum at first date) and ₹1000 monthly SIP (first-of-month nav)
growth_records = []
lumpsum_series_map = {}
sip_series_map = {}

for fund, g in df.groupby('fund_name'):
    g = g.sort_values('date').copy().reset_index(drop=True)
    if g.empty:
        continue

    # Lumpsum ₹1000 at first available nav
    first_nav = g['nav'].iloc[0]
    units = 1000.0 / first_nav
    g['lumpsum_value'] = units * g['nav']

    # Monthly SIP ₹1000 (use first available NAV of each calendar month)
    g['month'] = g['date'].dt.to_period('M')
    first_of_month = g.groupby('month').first().reset_index()
    first_of_month = first_of_month.set_index(pd.to_datetime(first_of_month['date']))['nav']
    if len(first_of_month) == 0:
        sip_value_series = pd.Series(dtype=float)
    else:
        sip_value_series, units_each, cum_units = compute_sip_units(first_of_month, sip_amount=1000)
        # create a full-date series (aligned to original dates): forward-fill the last known SIP value to all daily rows
        sip_value_daily = sip_value_series.reindex(g['date']).ffill().fillna(method='ffill')
        # If reindex fails due to mismatch, fallback to mapping by month end
        g['sip_value'] = sip_value_daily.values if len(sip_value_daily) == len(g) else np.nan

    lumpsum_series_map[fund] = g[['date','lumpsum_value']]
    if 'sip_value' in g:
        sip_series_map[fund] = g[['date','sip_value']]

    growth_records.append({
        'fund_name': fund,
        'first_date': g['date'].iloc[0],
        'last_date': g['date'].iloc[-1],
        'lumpsum_start_nav': first_nav,
        'lumpsum_end_value': float(g['lumpsum_value'].iloc[-1]),
        'sip_end_value': float(g['sip_value'].iloc[-1]) if 'sip_value' in g and not np.isnan(g['sip_value'].iloc[-1]) else np.nan
    })

growth_df = pd.DataFrame(growth_records).set_index('fund_name')
# If you want to inspect series for one fund:
# display(lumpsum_series_map[first_fund].tail())
# display(sip_series_map.get(first_fund).tail())


C:\Users\lenovo\AppData\Local\Temp\ipykernel_26872\3024245943.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sip_value_daily = sip_value_series.reindex(g['date']).ffill().fillna(method='ffill')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_26872\3024245943.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sip_value_daily = sip_value_series.reindex(g['date']).ffill().fillna(method='ffill')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_26872\3024245943.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sip_value_daily = sip_value_series.reindex(g['date']).ffill().fillna(method='ffill')
C:\Users\lenovo\AppData\Local\Temp\ipykernel_26872\3024245943.py:25: FutureWarning: Series.fillna with 'method' is deprecated and wil

In [11]:
print("Growth comparison (₹1000) — sample funds:")
display(growth_df.head(10))

Growth comparison (₹1000) — sample funds:


,first_date,last_date,lumpsum_start_nav,lumpsum_end_value,sip_end_value
fund_name,,,,,
DSP Midcap Fund,2013-01-02,2025-10-29,20.5420,8167.364424,517543.975045
HDFC Corporate Bond Fund,2013-01-01,2025-10-29,12.4504,2730.514682,258232.288799
HDFC Large and Mid Cap Fund,2014-06-30,2025-10-29,82.1250,4527.001522,378648.864627
ICICI Prudential Balanced Advantage Fund,2013-01-02,2025-10-29,17.4000,4935.632184,368927.696920
ICICI Prudential Large Cap Fund (erstwhile Bluechip Fund),2013-01-02,2025-10-29,18.6600,6770.632369,463518.211815
Nippon India Small Cap Fund,2013-01-02,2025-10-29,11.0142,17561.547820,858227.850390
SBI Large & Midcap Fund,2013-01-02,2025-10-29,89.1100,7832.772977,504716.554727
UTI Nifty 50 Index Fund,2013-01-02,2025-10-29,37.4040,4864.840659,380246.853692


In [12]:
sharpe_df.reset_index().to_csv("../data/processed/sharpe.csv", index=False)
sortino_df.reset_index().to_csv("../data/processed/sortino.csv", index=False)